In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation


In [32]:
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
import pandas as pd
import numpy as np
from surprise import accuracy, Dataset, SVD,SVDpp, NMF
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split as train_test_split_surprise
from tqdm.notebook import tqdm

import numpy as np
from sklearn.metrics import ndcg_score
import scipy.stats as ss

In [95]:
import sys
from os.path import dirname
sys.path.append(dirname("../../"))

from src.utilities import *
from src.reccomendation_util import *

In [4]:
%load_ext autoreload
%autoreload 2

### Load Data and Get Predictions

In [18]:
### Load DataSet

recipe_metadata = pd.read_csv("../../data/recommendation/RAW_recipes.csv")
print(recipe_metadata.shape)

raw_interactions = pd.read_csv("../../data/recommendation/RAW_interactions.csv")
print(raw_interactions.shape)

test_set = pd.read_csv("../../data/recommendation/ad_interaction_test.csv")
print(test_set.shape)

raw_subset= pd.read_csv("../../data/reccomendation_subset_data.csv")
print(raw_subset.shape)

### Subset Test Data

test_subset = test_set[test_set["user_id"].isin(set(raw_subset["user_id"]))]
print(test_subset.shape)

(231637, 13)
(1132367, 5)
(244884, 5)
(872021, 5)
(174103, 5)


In [74]:
(test_subset.user_id.value_counts()>1).mean()

0.7467205346900094

In [19]:
### Load Models
model1 = load_pickle("../../models/reccomender_model1_svd.pkl")
model2 = load_pickle("../../models/reccomender_model2_nmf.pkl")

### get predictions for a user
reader = Reader(rating_scale=(1, 5))

test_set_surprise_subset = Dataset.load_from_df(test_subset[["user_id", "recipe_id", "rating"]], reader)
_, test_set_surprise_subset = train_test_split_surprise(test_set_surprise_subset, test_size=1.0)

predictions1_sub = model1.test(test_set_surprise_subset)
predictions2_sub = model2.test(test_set_surprise_subset)

### Predicted Rank

In [20]:
prediction_df1 = pd.DataFrame(predictions1_sub)
prediction_df2 = pd.DataFrame(predictions2_sub)

pred_rank1 = get_prediction_ranking(prediction_df1)
pred_rank2 = get_prediction_ranking(prediction_df2)

In [97]:
prediction_df1.loc[(prediction_df1.uid==1533)].sort_values(by=["est"],ascending=False)

,uid,iid,r_ui,est,details
97235,1533,57549,5.0,4.984623,{'was_impossible': False}
58425,1533,44459,5.0,4.979205,{'was_impossible': False}
41186,1533,31048,5.0,4.909426,{'was_impossible': False}
112091,1533,10554,5.0,4.904533,{'was_impossible': False}
125476,1533,8857,5.0,4.799951,{'was_impossible': False}
44025,1533,41707,5.0,4.776423,{'was_impossible': False}
17273,1533,59135,5.0,4.772833,{'was_impossible': False}
48982,1533,13813,5.0,4.747607,{'was_impossible': False}
66818,1533,23891,5.0,4.746753,{'was_impossible': False}
14686,1533,57283,5.0,4.743313,{'was_impossible': False}


### True Rank

In [27]:
true_rank_baseline = get_true_ranking_user_baseline(test_subset)
true_rank_baseline.head()

,user_id,recipe_id,rating
0,1533,"[57283, 65352, 26988, 99394, 23891, 59135, 150...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
1,1535,"[107396, 297780, 153, 321620, 97952, 5483, 111...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
2,1634,"[8709, 28816, 361216, 16193, 132011, 9115, 262...","[5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 0, 0, 0]"
3,1676,"[126128, 136683, 7913, 204082, 36561, 825, 179...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 3]"
4,1792,"[32844, 405535, 31020, 91105, 88836, 87544, 89...","[5, 5, 5, 5, 5, 4, 4, 4, 3, 3]"


### NDCG: Evaluation Baseline

In [46]:
def ad_rank(x):
    rank_x = ss.rankdata(x,method="min").astype("int")
    rank_x = max(rank_x) - rank_x + 1
    return rank_x

In [70]:
def ndcg_overall(true_order,true_scores,pred_order,pred_scores, k=-1):
    true_rank = true_scores
    pred_rank = [pred_scores[pred_order.index(recipe)] for recipe in true_order]
    if len(true_order)==1 and len(set(true_order)-set(pred_order))==0:
        return 1.0
    if k>0:
        return ndcg_score(np.asarray([true_rank]),y_score = np.asarray([pred_rank]), k = k)
    else:
        return ndcg_score(np.asarray([true_rank]),y_score = np.asarray([pred_rank]))

In [65]:
rank_merged1 = pred_rank1.merge(true_rank_baseline,left_on="uid",right_on="user_id")
rank_merged2 = pred_rank2.merge(true_rank_baseline,left_on="uid",right_on="user_id")
rank_merged1.shape,rank_merged2.shape

((20049, 6), (20049, 6))

In [66]:
rank_merged1.head()

,uid,iid,est,user_id,recipe_id,rating
0,1533,"[57549, 44459, 31048, 10554, 8857, 41707, 5913...","[4.984623436267778, 4.979205256134577, 4.90942...",1533,"[57283, 65352, 26988, 99394, 23891, 59135, 150...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
1,1535,"[79278, 307072, 155496, 384168, 5483, 69914, 2...","[4.887660730957626, 4.869646919374931, 4.85833...",1535,"[107396, 297780, 153, 321620, 97952, 5483, 111...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
2,1634,"[132011, 11641, 28851, 44432, 9115, 12713, 120...","[3.830791080322248, 3.8093206066643965, 3.8043...",1634,"[8709, 28816, 361216, 16193, 132011, 9115, 262...","[5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 0, 0, 0]"
3,1676,"[126128, 825, 34233, 7913, 99156, 4022, 165679...","[4.914768923312829, 4.8161532228860695, 4.7314...",1676,"[126128, 136683, 7913, 204082, 36561, 825, 179...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 3]"
4,1792,"[87440, 88836, 88497, 91105, 111459, 32844, 89...","[5.0, 4.796666814631808, 4.604151827864249, 4....",1792,"[32844, 405535, 31020, 91105, 88836, 87544, 89...","[5, 5, 5, 5, 5, 4, 4, 4, 3, 3]"


In [75]:
rank_merged1["ndcg_overall"] = rank_merged1.apply(lambda row:\
                                                  ndcg_overall(row.recipe_id,row.rating,\
                                                              row.iid,row.est),axis=1)

rank_merged2["ndcg_overall"] = rank_merged2.apply(lambda row:\
                                                  ndcg_overall(row.recipe_id,row.rating,\
                                                              row.iid,row.est),axis=1)

rank_merged1["ndcg_10"] = rank_merged1.apply(lambda row:\
                                                  ndcg_overall(row.recipe_id,row.rating,\
                                                              row.iid,row.est,10),axis=1)

rank_merged2["ndcg_10"] = rank_merged2.apply(lambda row:\
                                                  ndcg_overall(row.recipe_id,row.rating,\
                                                              row.iid,row.est,10),axis=1)

In [83]:
rank_merged1[["ndcg_overall","ndcg_10"]].describe()

,ndcg_overall,ndcg_10
count,20049.000000,20049.000000
mean,0.976004,0.970740
std,0.076782,0.081557
min,0.000000,0.000000
25%,0.985902,0.977991
50%,1.000000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [84]:
rank_merged2[["ndcg_overall","ndcg_10"]].describe()

,ndcg_overall,ndcg_10
count,20049.000000,20049.000000
mean,0.974047,0.967682
std,0.070703,0.075987
min,0.000000,0.000000
25%,0.975388,0.966667
50%,1.000000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [85]:
rank_merged1.loc[rank_merged1.recipe_id.apply(lambda x: len(x)>10)][["ndcg_overall","ndcg_10"]].describe()

,ndcg_overall,ndcg_10
count,2730.000000,2730.000000
mean,0.975078,0.936421
std,0.037858,0.074996
min,0.000000,0.000000
25%,0.967594,0.907721
50%,0.987314,0.960247
75%,0.995977,0.987276
max,1.000000,1.000000


In [86]:
rank_merged2.loc[rank_merged2.recipe_id.apply(lambda x: len(x)>10)][["ndcg_overall","ndcg_10"]].describe()

,ndcg_overall,ndcg_10
count,2730.000000,2730.000000
mean,0.970257,0.923517
std,0.034633,0.068204
min,0.000000,0.000000
25%,0.960363,0.892308
50%,0.978439,0.938462
75%,0.990948,0.973293
max,1.000000,1.000000


#### MAP

In [5]:
def get_ndcg(a,b,k=-1):
    true_rank = list(range(1,len(a)+1))
    pred_rank = [b.index(true_item)+1 for true_item in a]
    if len(a)==1 and len(set(a)-set(b))==0:
        return 1
    if k>0:
        return ndcg_score(np.asarray([true_rank]),y_score = np.asarray([pred_rank]), k = k)
    else:
        return ndcg_score(np.asarray([true_rank]),y_score = np.asarray([pred_rank]))